### Init ray

In [1]:
import ray
ray.init(address="auto", ignore_reinit_error=True)

2020-12-06 18:44:09,912	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379


{'node_ip_address': '192.168.8.51',
 'raylet_ip_address': '192.168.8.51',
 'redis_address': '192.168.8.51:6379',
 'object_store_address': '/tmp/ray/session_2020-12-04_17-50-59_110223_23718/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-04_17-50-59_110223_23718/sockets/raylet',
 'webui_url': '192.168.8.51:8265',
 'session_dir': '/tmp/ray/session_2020-12-04_17-50-59_110223_23718',
 'metrics_export_port': 57837}

### Load default config

In [2]:
# load defaulft config
import yaml
import os
import time
import pandas as pd
import random

config_path = './configs/Active_v0.yml'

with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

# create base dir and gr
if os.path.exists(config["PROJECT"]["project_dir"]) is False:
    os.mkdir(config["PROJECT"]["project_dir"])

if os.path.exists(config["PROJECT"]["group_dir"]) is False:
    os.mkdir(config["PROJECT"]["group_dir"])

In [3]:
config

{'ACTIVE_ALGO': {'LOSSLEARNING': 1.0},
 'DATASET': {'height_shift_range': 4,
  'horizontal_flip': True,
  'width_shift_range': 4},
 'NETWORK': {'CLASSES': None,
  'INPUT_SIZE': None,
  'MARGIN': 1.0,
  'embedding_size': 128},
 'PROJECT': {'Backbone': 'resnet18',
  'dataset_name': 'CIFAR',
  'group': 'Active_Learning_v1',
  'group_dir': '/disk3/Active_Learning_test/Active_Learning_v1',
  'project': 'Active_Learning_CIFAR',
  'project_dir': '/disk3/Active_Learning_test',
  'source': 'CIFAR'},
 'RUNS': {'ADDENDUM': 1000,
  'CYCLES': 10,
  'SUBSET': -1,
  'TRIALS': 1,
  'test_each': 5},
 'TEST': {'batch_size': 128},
 'TRAIN': {'Data_augementation': True,
  'EPOCH_SLIT': 80,
  'EPOCH_WARMUP': 2,
  'EPOCH_WHOLE': 120,
  'MILESTONES': [160],
  'batch_size': 128,
  'gamma': 0.1,
  'lr': 0.001,
  'start_epoch': 0,
  'transfer_weight_path': False,
  'w_c_loss': 1.0,
  'w_l_loss': 1.0,
  'wdecay': 0.9995}}

### Get the data to annotate

For DataNet we should pass the list of the annotated images

In [4]:
# Get the data to annotate

#############################################################################################
# LOAD DATA
#############################################################################################
from data_utils import CIFAR10Data
# Load data
cifar10_data = CIFAR10Data()
num_classes = len(cifar10_data.classes)
x_train, y_train, x_test, y_test = cifar10_data.get_data(subtract_mean=True)

/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/reminiz/ReminizML2/python_env/lib/pyt

CIFAR10 Training data shape: (50000, 32, 32, 3)
CIFAR10 Training label shape (50000, 1)
CIFAR10 Test data shape (10000, 32, 32, 3)
CIFAR10 Test label shape (10000, 1)


In [5]:
indices = list(range(len(x_train)))
random.shuffle(indices)
labeled_set = indices[:config["RUNS"]["ADDENDUM"] ]
unlabeled_set = indices[config["RUNS"]["ADDENDUM"] :]

In [6]:
# test with all the images
NUM_IMAGES_TEST = len(x_test)
# Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
test_set = list(range(NUM_IMAGES_TEST))

In [7]:
config["NETWORK"]["INPUT_SIZE"] =  x_train[0].shape[0]
config["NETWORK"]["CLASSES"] = cifar10_data.classes

In [8]:
config

{'ACTIVE_ALGO': {'LOSSLEARNING': 1.0},
 'DATASET': {'height_shift_range': 4,
  'horizontal_flip': True,
  'width_shift_range': 4},
 'NETWORK': {'CLASSES': ['plane',
   'car',
   'bird',
   'cat',
   'deer',
   'dog',
   'frog',
   'horse',
   'ship',
   'truck'],
  'INPUT_SIZE': 32,
  'MARGIN': 1.0,
  'embedding_size': 128},
 'PROJECT': {'Backbone': 'resnet18',
  'dataset_name': 'CIFAR',
  'group': 'Active_Learning_v1',
  'group_dir': '/disk3/Active_Learning_test/Active_Learning_v1',
  'project': 'Active_Learning_CIFAR',
  'project_dir': '/disk3/Active_Learning_test',
  'source': 'CIFAR'},
 'RUNS': {'ADDENDUM': 1000,
  'CYCLES': 10,
  'SUBSET': -1,
  'TRIALS': 1,
  'test_each': 5},
 'TEST': {'batch_size': 128},
 'TRAIN': {'Data_augementation': True,
  'EPOCH_SLIT': 80,
  'EPOCH_WARMUP': 2,
  'EPOCH_WHOLE': 120,
  'MILESTONES': [160],
  'batch_size': 128,
  'gamma': 0.1,
  'lr': 0.001,
  'start_epoch': 0,
  'transfer_weight_path': False,
  'w_c_loss': 1.0,
  'w_l_loss': 1.0,
  'wdecay':

In [9]:
from train_agent_cifar_keras import Active_Learning_train

In [10]:
NetworkActor =  Active_Learning_train.remote(config, labeled_set, test_set,  0, False)

(pid=17741) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17741)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=17741) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17741)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=17741) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17741)   _np_qint16 = np.dtyp

(pid=17741) CIFAR10 Training data shape: (50000, 32, 32, 3)
(pid=17741) CIFAR10 Training label shape (50000, 1)
(pid=17741) CIFAR10 Test data shape (10000, 32, 32, 3)
(pid=17741) CIFAR10 Test label shape (10000, 1)


(pid=17741) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:429: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
(pid=17741) Instructions for updating:
(pid=17741) tf.py_func is deprecated in TF V2. Instead, use
(pid=17741)     tf.py_function, which takes a python function which manipulates tf eager
(pid=17741)     tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
(pid=17741)     an ndarray (just call tensor.numpy()) but having access to eager tensors
(pid=17741)     means `tf.py_function`s can use accelerators such as GPUs as well as
(pid=17741)     being differentiable using a gradient tape.
(pid=17741)     
(pid=17741) WARNING:tensorflow:From /mnt/Ressources/Andres/Temp_active/backbones/resnet18.py:22: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
(pid=17741) Ins

(pid=22622, ip=192.168.8.52) Epoch: 15 Time: 2020-12-06 19:04:30 Train loss: 2.31 Count NaN: 0 Saving


(pid=30463, ip=192.168.8.52) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=30463, ip=192.168.8.52)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=30463, ip=192.168.8.52) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=30463, ip=192.168.8.52)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=30463, ip=192.168.8.52) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, i

...

(pid=3590) Train_Stage_0 => Starts to train from scratch 
(pid=3590) Train_Stage_0 self.steps_per_epoch 8


(pid=3590) 2020-12-06 20:10:36.645472: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally


(pid=3590) Train_Stage_0 Epoch:  0 / 201 || Time: 2020-12-06 20:10:39 || Accuracy: 13.03 || MAE LossNet: 8.07 || 
(pid=3590) Train_Stage_0 Epoch:  1 / 201 || Time: 2020-12-06 20:10:41 || Accuracy: 17.27 || MAE LossNet: 11.56 || 
(pid=3590) Train_Stage_0 Epoch:  2 / 201 || Time: 2020-12-06 20:10:42 || Accuracy: 20.14 || MAE LossNet: 7.36 || 
(pid=3590) Train_Stage_0 Epoch:  3 / 201 || Time: 2020-12-06 20:10:43 || Accuracy: 27.00 || MAE LossNet: 4.46 || 
(pid=3590) Train_Stage_0 Epoch:  4 / 201 || Time: 2020-12-06 20:10:44 || Accuracy: 27.10 || MAE LossNet: 2.50 || 
(pid=3590) Train_Stage_0 Epoch:  5 / 201 || Time: 2020-12-06 20:10:45 || Accuracy: 28.06 || MAE LossNet: 2.86 || 
(pid=3590) Train_Stage_0 Epoch:  6 / 201 || Time: 2020-12-06 20:10:46 || Accuracy: 31.81 || MAE LossNet: 2.75 || 
(pid=3590) Train_Stage_0 Epoch:  7 / 201 || Time: 2020-12-06 20:10:47 || Accuracy: 36.31 || MAE LossNet: 2.14 || 
(pid=3590) Train_Stage_0 Epoch:  8 / 201 || Time: 2020-12-06 20:10:47 || Accuracy: 38.4

(pid=3590) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/training/saver.py:966: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
(pid=3590) Instructions for updating:
(pid=3590) Use standard file APIs to delete files with this prefix.


(pid=3590) Train_Stage_0 Epoch: 51 / 201 || Time: 2020-12-06 20:11:29 || Accuracy: 86.70 || MAE LossNet: 1.19 || 
(pid=3590) Train_Stage_0 Epoch: 52 / 201 || Time: 2020-12-06 20:11:30 || Accuracy: 85.74 || MAE LossNet: 1.14 || 
(pid=3590) Train_Stage_0 Epoch: 53 / 201 || Time: 2020-12-06 20:11:31 || Accuracy: 85.70 || MAE LossNet: 1.22 || 
(pid=3590) Train_Stage_0 Epoch: 54 / 201 || Time: 2020-12-06 20:11:32 || Accuracy: 86.52 || MAE LossNet: 1.14 || 
(pid=3590) Train_Stage_0 Epoch: 55 / 201 || Time: 2020-12-06 20:11:32 || Accuracy: 88.28 || MAE LossNet: 1.12 || 
(pid=3590) Train_Stage_0 Epoch: 56 / 201 || Time: 2020-12-06 20:11:33 || Accuracy: 88.48 || MAE LossNet: 1.23 || 
(pid=3590) Train_Stage_0 Epoch: 57 / 201 || Time: 2020-12-06 20:11:34 || Accuracy: 90.05 || MAE LossNet: 1.24 || 
(pid=3590) Train_Stage_0 Epoch: 58 / 201 || Time: 2020-12-06 20:11:35 || Accuracy: 91.01 || MAE LossNet: 1.11 || 
(pid=3590) Train_Stage_0 Epoch: 59 / 201 || Time: 2020-12-06 20:11:36 || Accuracy: 88.41

### Set Configuration from the data

In [ ]:
from train_agent_cifar import Active_Learning_train
from inference_agent_cifar import Active_Learning_inference

In [ ]:
num_run =0

In [ ]:
initial_weight_path = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(num_run-1),'checkpoint','epoch200.ckpt-200')

In [ ]:
initial_weight_path

In [ ]:
num_run = 0
for num_run in range(6,10):
    if num_run==0:
        initial_weight_path = False
    else:
        initial_weight_path = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(num_run-1),'checkpoint','epoch200.ckpt-200')
        
        
    NetworkActor =  Active_Learning_train.remote(config, labeled_set, test_set,  num_run, initial_weight_path)
    NetworkActor.start_training.remote()
    
    # Wait util the model is training
    while True:
        time.sleep(10)
        try:
            progress_id = NetworkActor.isTraining.remote()
            response = ray.get(progress_id)
            break
        except:
            pass
        
    # wait until the model finish training
    while True:
        time.sleep(10)
        progress_id = NetworkActor.isTraining.remote()
        response = ray.get(progress_id)
        if not response:
            break
    
    NetworkActor.__ray_terminate__.remote()
    
    del NetworkActor
    
    if config["TRAIN"]["w_l_loss"] > 0:
        
        weight_file = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(num_run),'checkpoint','epoch200.ckpt-200')
        
        AL_inference = Active_Learning_inference.remote( config, unlabeled_set, num_run, weight_file)
        AL_inference.evaluate.remote()

        run_dir   = os.path.join(config["PROJECT"]["group_dir"],"Stage_"+str(num_run))
        ordered_indexes   = os.path.join(run_dir, "ordered_indexes.csv")

        # wait the file qith the scores is generated
        while True:
            time.sleep(10)
            if os.path.isfile(ordered_indexes):
                break

        # read the scores file and create the new labeled set and unlabeled set to repeat the trainig
        pd_ordered_indexes = pd.read_csv(ordered_indexes)
        new_annotated_data = list(pd_ordered_indexes.iloc[:config["RUNS"]["ADDENDUM"]]['indexes'].to_numpy())
        labeled_set += new_annotated_data
        unlabeled_set =  list(pd_ordered_indexes.iloc[config["RUNS"]["ADDENDUM"]:]['indexes'].to_numpy())
    else :
        num_images = (num_run+2)*config["RUNS"]["ADDENDUM"]
        labeled_set = indices[: num_images]
        unlabeled_set = indices[num_images :]